# Modelado — Próximo pedido **DIGITAL** (v3.2, PySpark LR + Desbalance + Backtesting + PR@k)

**Fecha de generación del cuaderno:** 2025-09-16

**Objetivo:** predecir si el **próximo pedido** de un cliente será **DIGITAL**, optimizando **F1** y **PR@k** (métricas útiles para campañas).  
**Enfoque:**
- Modelo central: **Regresión Logística (LR)** (estable, explicable, rápido).  
- **Estrategias de desbalance** sobre el *train*: `weightCol`, **oversampling**, **undersampling**.  
- **Backtesting** en múltiples cortes temporales.  
- **Tuning de umbral** para maximizar F1 y cálculo de **Precision/Recall@k** y **Lift@k**.

> Nota: Dejamos *ganchos* para integrar LightGBM/XGBoost si tu entorno lo permite. En Spark 4.0.1 puede requerir dependencias específicas; este notebook funciona 100% con PySpark estándar.

## 1) Sesión Spark (config estable)
- Ejecuta en `local[*]`.
- Activa **Kryo** y sube memoria para evitar OOM.
- Ajusta `DATA_DIR` según tu ruta.

In [1]:
from pyspark.sql import SparkSession, functions as F, types as T, Window
from pyspark import StorageLevel
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, Imputer, VectorIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder
from pyspark.ml.functions import vector_to_array
import math

try:
    spark.stop()
except Exception:
    pass

spark = (
    SparkSession.builder
    .appName("modelado-proximo-pedido-digital-v3.2-lr-imbalance-backtest")
    .master("local[*]")
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")
    .config("spark.driver.memory", "12g")
    .config("spark.sql.shuffle.partitions", "200")
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
    .config("spark.kryoserializer.buffer", "32m")
    .config("spark.kryoserializer.buffer.max", "512m")
    .config("spark.sql.warehouse.dir", "./spark-warehouse")
    .getOrCreate()
)
spark.sparkContext.setLogLevel("WARN")
spark.sparkContext.setCheckpointDir("/tmp/spark_chk")

DATA_DIR = "dataset/dataset"   # <-- ajusta si tu ruta cambia
DEFAULT_TEST_START_YM = "2024-01"  # corte por defecto
BACKTEST_SPLITS = ["2023-08", "2023-10", "2023-12", "2024-01"]  # puedes editar
print("Spark version:", spark.version)


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/09/17 07:06:38 WARN Utils: Your hostname, debian, resolves to a loopback address: 127.0.1.1; using 192.168.1.43 instead (on interface wlo1)
25/09/17 07:06:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/17 07:06:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark version: 4.0.1


## 2) Carga y preparación mínima
- Derivamos `month_first`, `ym`, `is_digital`.
- Mantén tipos simples y nombres consistentes.

In [2]:
df = spark.read.parquet(DATA_DIR)

df = (df
    .withColumn("month_first", F.trunc("fecha_pedido_dt", "month"))
    .withColumn("ym", F.date_format("month_first", "yyyy-MM"))
    .withColumn("is_digital", F.when(F.col("canal_pedido_cd")=="DIGITAL", 1).otherwise(0))
)

df.select("cliente_id","fecha_pedido_dt","ym","canal_pedido_cd","is_digital").show(5, truncate=False)


+----------+-------------------+-------+---------------+----------+
|cliente_id|fecha_pedido_dt    |ym     |canal_pedido_cd|is_digital|
+----------+-------------------+-------+---------------+----------+
|C089085   |2023-05-16 19:00:00|2023-05|VENDEDOR       |0         |
|C073952   |2023-10-06 19:00:00|2023-10|VENDEDOR       |0         |
|C101443   |2023-01-04 19:00:00|2023-01|DIGITAL        |1         |
|C055939   |2024-01-12 19:00:00|2024-01|DIGITAL        |1         |
|C088826   |2023-09-30 19:00:00|2023-09|DIGITAL        |1         |
+----------+-------------------+-------+---------------+----------+
only showing top 5 rows


## 3) Etiqueta por **cliente-mes** (sin fuga)
- Tomamos el **último pedido** del mes por cliente.
- `label` = si el **próximo pedido** del cliente es DIGITAL.

In [3]:
all_cols = df.columns
w_client_order = Window.partitionBy("cliente_id").orderBy(F.col("fecha_pedido_dt").asc(),
                                                          F.hash(*[F.col(c) for c in all_cols]).asc())
w_client_month_desc = Window.partitionBy("cliente_id","month_first").orderBy(F.col("fecha_pedido_dt").desc(),
                                                                             F.hash(*[F.col(c) for c in all_cols]).desc())

orders = (df
    .withColumn("prev_dt", F.lag("fecha_pedido_dt").over(w_client_order))
    .withColumn("next_canal", F.lead("canal_pedido_cd").over(w_client_order))
    .withColumn("next_is_digital", F.when(F.col("next_canal")=="DIGITAL", 1).otherwise(0))
    .withColumn("recency_days", F.datediff(F.col("fecha_pedido_dt"), F.col("prev_dt")))
    .withColumn("rn_month_desc", F.row_number().over(w_client_month_desc))
)

last_in_month = (orders
    .filter(F.col("rn_month_desc")==1)
    .select("cliente_id","month_first","ym",
            F.col("recency_days").alias("recency_days_last"),
            F.col("next_is_digital").alias("label"))
)
last_in_month.show(5, truncate=False)


+----------+-----------+-------+-----------------+-----+
|cliente_id|month_first|ym     |recency_days_last|label|
+----------+-----------+-------+-----------------+-----+
|C000009   |2023-01-01 |2023-01|NULL             |0    |
|C000009   |2023-08-01 |2023-08|213              |0    |
|C000009   |2023-11-01 |2023-11|4                |0    |
|C000009   |2023-12-01 |2023-12|8                |1    |
|C000009   |2024-01-01 |2024-01|28               |1    |
+----------+-----------+-------+-----------------+-----+
only showing top 5 rows


## 4) Features (RFM + rolling + priors + ciclo de vida)
- Agregados por cliente-mes, ratios por segmento, rolling 3m, crecimiento, etc.

In [4]:
monthly_agg = (df.groupBy("cliente_id","month_first","ym")
    .agg(
        F.count("*").alias("n_orders"),
        F.avg("is_digital").alias("digital_ratio"),
        F.sum(F.col("facturacion_usd_val").cast("double")).alias("sum_fact"),
        F.avg(F.col("facturacion_usd_val").cast("double")).alias("avg_fact"),
        F.sum(F.col("cajas_fisicas").cast("double")).alias("sum_cajas"),
        F.avg(F.col("cajas_fisicas").cast("double")).alias("avg_cajas"),
        F.avg(F.col("materiales_distintos_val").cast("double")).alias("avg_mat_dist"),
        F.first("tipo_cliente_cd", ignorenulls=True).alias("tipo_cliente_cd"),
        F.first("madurez_digital_cd", ignorenulls=True).alias("madurez_digital_cd"),
        F.first("frecuencia_visitas_cd", ignorenulls=True).alias("frecuencia_visitas_cd"),
        F.first("pais_cd", ignorenulls=True).alias("pais_cd"),
        F.first("region_comercial_txt", ignorenulls=True).alias("region_comercial_txt")
    )
)

w_client_month = Window.partitionBy("cliente_id").orderBy(F.col("month_first").asc())
first_month = (monthly_agg
               .withColumn("first_month", F.first("month_first", ignorenulls=True).over(w_client_month))
               .select("cliente_id","first_month").distinct())
monthly_agg = (monthly_agg
               .join(first_month, on="cliente_id", how="left")
               .withColumn("months_since_first", F.floor(F.months_between("month_first", "first_month"))))

region_month = (df.groupBy("region_comercial_txt","month_first").agg(F.avg("is_digital").alias("region_digital_ratio")))
region_month = region_month.withColumn("ym", F.date_format("month_first", "yyyy-MM"))
w_region = Window.partitionBy("region_comercial_txt").orderBy(F.col("month_first").asc())
region_month = (region_month
                .withColumn("region_digital_ratio_lag1", F.lag("region_digital_ratio", 1).over(w_region))
                .select("region_comercial_txt","ym","region_digital_ratio_lag1"))

tipo_month = (df.groupBy("tipo_cliente_cd","month_first").agg(F.avg("is_digital").alias("tipo_digital_ratio")))
tipo_month = tipo_month.withColumn("ym", F.date_format("month_first", "yyyy-MM"))
w_tipo = Window.partitionBy("tipo_cliente_cd").orderBy(F.col("month_first").asc())
tipo_month = (tipo_month
              .withColumn("tipo_digital_ratio_lag1", F.lag("tipo_digital_ratio", 1).over(w_tipo))
              .select("tipo_cliente_cd","ym","tipo_digital_ratio_lag1"))

w_roll3 = w_client_month.rowsBetween(-3, -1)
ds = (monthly_agg
    .join(last_in_month, on=["cliente_id","month_first","ym"], how="left")
    .withColumn("lag1_digital_ratio", F.lag("digital_ratio", 1).over(w_client_month))
    .withColumn("n_orders_3m", F.sum("n_orders").over(w_roll3))
    .withColumn("digital_ratio_3m", F.avg("digital_ratio").over(w_roll3))
    .withColumn("sum_fact_3m", F.sum("sum_fact").over(w_roll3))
    .withColumn("growth_digital_ratio", F.col("digital_ratio") - F.col("lag1_digital_ratio"))
    .join(region_month, on=["region_comercial_txt","ym"], how="left")
    .join(tipo_month, on=["tipo_cliente_cd","ym"], how="left")
    .filter(F.col("label").isNotNull())
)
ds.persist(StorageLevel.MEMORY_AND_DISK)
print("Rows ds:", ds.count())


25/09/17 07:06:48 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Rows ds: 1022849


## 5) Utilidades: desbalance, PR@k/Recall@k, F1 vs threshold

In [5]:
def apply_class_weights(df_train, label_col="label"):
    pos = df_train.filter(F.col(label_col)==1).count()
    neg = df_train.filter(F.col(label_col)==0).count()
    ratio = (neg / float(max(pos,1))) if pos else 1.0
    return df_train.withColumn("weight", F.when(F.col(label_col)==1, F.lit(ratio)).otherwise(F.lit(1.0)))

def oversample_minority(df_train, label_col="label", target_ratio=0.5):
    # target_ratio = proporción de positivos deseada (ej: 0.5 => 1:1)
    counts = df_train.groupBy(label_col).count().collect()
    cnt = {int(r[label_col]): r['count'] for r in counts}
    pos, neg = cnt.get(1,0), cnt.get(0,0)
    if pos==0 or neg==0:
        return df_train
    current_ratio = pos / float(pos+neg)
    if current_ratio >= target_ratio:
        return df_train
    desired_pos = int(math.ceil(target_ratio * (pos+neg) / (1 - target_ratio)))  # algebra inversa
    add_pos = max(desired_pos - pos, 0)
    if add_pos <= 0:
        return df_train
    # sampling con reemplazo para positivos
    frac = add_pos / float(pos)
    df_pos = df_train.filter(F.col(label_col)==1)
    df_pos_extra = df_pos.sample(withReplacement=True, fraction=frac, seed=42)
    return df_train.unionByName(df_pos_extra)

def undersample_majority(df_train, label_col="label", target_ratio=0.5):
    counts = df_train.groupBy(label_col).count().collect()
    cnt = {int(r[label_col]): r['count'] for r in counts}
    pos, neg = cnt.get(1,0), cnt.get(0,0)
    if pos==0 or neg==0:
        return df_train
    desired_neg = int((pos * (1 - target_ratio)) / max(target_ratio, 1e-6))
    keep_neg = max(min(desired_neg, neg), 1)
    frac = keep_neg / float(neg)
    df_neg = df_train.filter(F.col(label_col)==0).sample(withReplacement=False, fraction=frac, seed=42)
    df_pos = df_train.filter(F.col(label_col)==1)
    return df_pos.unionByName(df_neg)

def precision_recall_at_k(pred_df, k=0.1, label_col="label", score_col="p1"):
    total = pred_df.count()
    k_n = max(int(total * k), 1)
    topk = pred_df.orderBy(F.col(score_col).desc()).limit(k_n)
    tp = topk.filter(F.col(label_col)==1).count()
    positives = pred_df.filter(F.col(label_col)==1).count()
    precision = tp / float(k_n)
    recall = tp / float(max(positives,1))
    # Lift@k = precision@k / base_rate
    base_rate = positives / float(max(total,1))
    lift = precision / float(max(base_rate,1e-9))
    return precision, recall, lift

def best_threshold_for_f1(pred_df, label_col="label", score_col="p1", grid_size=101):
    # Evalúa F1 en thresholds uniformes [0,1]
    best = (0.5, 0.0, 0.0, 0.0)  # thr, precision, recall, f1
    for i in range(grid_size):
        thr = i / float(grid_size-1)
        pred = pred_df.withColumn("pred", F.when(F.col(score_col)>=thr, F.lit(1)).otherwise(F.lit(0)))
        cm = pred.groupBy(label_col, "pred").count().toPandas()
        tp = int(cm[(cm[label_col]==1) & (cm["pred"]==1)]["count"].sum())
        tn = int(cm[(cm[label_col]==0) & (cm["pred"]==0)]["count"].sum())
        fp = int(cm[(cm[label_col]==0) & (cm["pred"]==1)]["count"].sum())
        fn = int(cm[(cm[label_col]==1) & (cm["pred"]==0)]["count"].sum())
        precision = tp / float(max(tp+fp,1))
        recall    = tp / float(max(tp+fn,1))
        f1 = (2*precision*recall) / float(max(precision+recall,1e-9))
        if f1 > best[3]:
            best = (thr, precision, recall, f1)
    return best


## 6) Pipeline LR + grids compactos
- OHE solo para **baja cardinalidad**.
- Imputación de numéricas.

In [6]:
num_cols = [
    "n_orders","digital_ratio","lag1_digital_ratio","sum_fact","avg_fact",
    "sum_cajas","avg_cajas","avg_mat_dist","recency_days_last",
    "n_orders_3m","digital_ratio_3m","sum_fact_3m","growth_digital_ratio",
    "months_since_first","region_digital_ratio_lag1","tipo_digital_ratio_lag1"
]

cat_low = ["madurez_digital_cd","frecuencia_visitas_cd","pais_cd","tipo_cliente_cd"]

imputer = Imputer(inputCols=num_cols, outputCols=[c+"_imp" for c in num_cols])
idxs = [StringIndexer(inputCol=c, outputCol=c+"_idx", handleInvalid="keep") for c in cat_low]
ohe = OneHotEncoder(inputCols=[c+"_idx" for c in cat_low], outputCols=[c+"_oh" for c in cat_low])
feats = [c+"_imp" for c in num_cols] + [c+"_oh" for c in cat_low]
asm = VectorAssembler(inputCols=feats, outputCol="features_lr")

lr = LogisticRegression(featuresCol="features_lr", labelCol="label", weightCol="weight",
                        maxIter=80, regParam=0.01, elasticNetParam=0.0)

pipe_lr = Pipeline(stages=[imputer] + idxs + [ohe, asm, lr])

grid_lr = (ParamGridBuilder()
           .addGrid(lr.regParam, [0.0, 0.01, 0.1])
           .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])
           .build())

e_pr = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderPR")
tvs_lr = TrainValidationSplit(estimator=pipe_lr, estimatorParamMaps=grid_lr, evaluator=e_pr, trainRatio=0.85, parallelism=1)


## 7) Entrenador con estrategia de desbalance
Parámetro `imbalance_strategy` puede ser: `"weights"` (default), `"oversample"`, `"undersample"`, `"none"`.

In [7]:
def train_lr_with_imbalance(train_df, strategy="weights"):
    df_tr = train_df
    if strategy == "weights":
        df_tr = apply_class_weights(df_tr)
    elif strategy == "oversample":
        df_tr = oversample_minority(df_tr)
        df_tr = df_tr.withColumn("weight", F.lit(1.0))
    elif strategy == "undersample":
        df_tr = undersample_majority(df_tr)
        df_tr = df_tr.withColumn("weight", F.lit(1.0))
    else:
        df_tr = df_tr.withColumn("weight", F.lit(1.0))

    df_tr = df_tr.checkpoint(eager=True)
    model = tvs_lr.fit(df_tr)
    return model


## 8) Evaluación: AUC PR, F1 (best-threshold), PR@k/Recall@k/Lift@k

In [8]:
def evaluate_predictions(pred, label_col="label"):
    pred = pred.withColumn("p1", vector_to_array("probability")[1]).cache()
    e_auc  = BinaryClassificationEvaluator(labelCol=label_col, rawPredictionCol="rawPrediction", metricName="areaUnderROC")
    e_aupr = BinaryClassificationEvaluator(labelCol=label_col, rawPredictionCol="rawPrediction", metricName="areaUnderPR")
    auc  = e_auc.evaluate(pred)
    aupr = e_aupr.evaluate(pred)
    thr, p_thr, r_thr, f1_thr = best_threshold_for_f1(pred.select(label_col, "p1"))
    p5, r5, l5 = precision_recall_at_k(pred.select(label_col, "p1"), k=0.05)
    p10, r10, l10 = precision_recall_at_k(pred.select(label_col, "p1"), k=0.10)
    metrics = {
        "auc_roc": auc, "auc_pr": aupr,
        "best_threshold": thr, "precision_at_best_thr": p_thr, "recall_at_best_thr": r_thr, "f1_at_best_thr": f1_thr,
        "precision@5%": p5, "recall@5%": r5, "lift@5%": l5,
        "precision@10%": p10, "recall@10%": r10, "lift@10%": l10,
    }
    return pred, metrics


## 9) Backtesting: múltiples cortes `TEST_START_YM`

In [9]:
def run_backtests(ds, test_splits, imbalance_strategy="weights"):
    results = []
    for cut in test_splits:
        print(f"\n=== Backtest corte TEST_START_YM = {cut} ===")
        train = ds.filter(F.col("ym") < F.lit(cut))
        test  = ds.filter(F.col("ym") >= F.lit(cut))
        print("Train rows:", train.count(), "| Test rows:", test.count())
        train = train.repartition(200).persist(StorageLevel.MEMORY_AND_DISK)
        _ = train.count()
        model = train_lr_with_imbalance(train, strategy=imbalance_strategy)
        pred = model.transform(test).cache()
        pred, met = evaluate_predictions(pred)
        met_row = {
            "cut": cut,
            **{k: round(float(v), 6) for k,v in met.items()}
        }
        results.append(met_row)
        print(met_row)
    return results

results_weights = run_backtests(ds, BACKTEST_SPLITS, imbalance_strategy="weights")



=== Backtest corte TEST_START_YM = 2023-08 ===
Train rows: 361852 | Test rows: 660997


{'cut': '2023-08', 'auc_roc': 0.62331, 'auc_pr': 0.499691, 'best_threshold': 0.0, 'precision_at_best_thr': 0.371971, 'recall_at_best_thr': 1.0, 'f1_at_best_thr': 0.542244, 'precision@5%': 0.579594, 'recall@5%': 0.077906, 'lift@5%': 1.558168, 'precision@10%': 0.579025, 'recall@10%': 0.155662, 'lift@10%': 1.556639}

=== Backtest corte TEST_START_YM = 2023-10 ===
Train rows: 465637 | Test rows: 557212


{'cut': '2023-10', 'auc_roc': 0.619602, 'auc_pr': 0.47347, 'best_threshold': 0.38, 'precision_at_best_thr': 0.446656, 'recall_at_best_thr': 0.632661, 'f1_at_best_thr': 0.523631, 'precision@5%': 0.549174, 'recall@5%': 0.077891, 'lift@5%': 1.557848, 'precision@10%': 0.547783, 'recall@10%': 0.155389, 'lift@10%': 1.5539}

=== Backtest corte TEST_START_YM = 2023-12 ===
Train rows: 569153 | Test rows: 453696


{'cut': '2023-12', 'auc_roc': 0.614559, 'auc_pr': 0.437458, 'best_threshold': 0.38, 'precision_at_best_thr': 0.412553, 'recall_at_best_thr': 0.632214, 'f1_at_best_thr': 0.499292, 'precision@5%': 0.510007, 'recall@5%': 0.07825, 'lift@5%': 1.565051, 'precision@10%': 0.506668, 'recall@10%': 0.155478, 'lift@10%': 1.554804}

=== Backtest corte TEST_START_YM = 2024-01 ===
Train rows: 621769 | Test rows: 401080


{'cut': '2024-01', 'auc_roc': 0.611897, 'auc_pr': 0.413518, 'best_threshold': 0.38, 'precision_at_best_thr': 0.389729, 'recall_at_best_thr': 0.632464, 'f1_at_best_thr': 0.482276, 'precision@5%': 0.482946, 'recall@5%': 0.078485, 'lift@5%': 1.569692, 'precision@10%': 0.479081, 'recall@10%': 0.155713, 'lift@10%': 1.557131}


## 10) Coeficientes (explicabilidad LR)
- Muestra los coeficientes del mejor modelo del último corte (como referencia).

In [12]:
from pyspark.ml.classification import LogisticRegressionModel

# Entrena una vez para extraer coeficientes sobre el último corte
cut = BACKTEST_SPLITS[-1] if BACKTEST_SPLITS else DEFAULT_TEST_START_YM
train = ds.filter(F.col("ym") < F.lit(cut))

model = train_lr_with_imbalance(train, strategy="weights")
best  = model.bestModel  # <- PipelineModel

# 1) Ubicar la etapa correcta (modelo ya entrenado)
stage_types = [type(s).__name__ for s in best.stages]
lr_stage = next((s for s in best.stages if isinstance(s, LogisticRegressionModel)), None)
if lr_stage is None:
    raise ValueError(f"No encontré LogisticRegressionModel en stages: {stage_types}")

# 2) Coeficientes e intercepto
coef = lr_stage.coefficients.toArray()
intercept = lr_stage.intercept
print("Intercept:", intercept)
print("Nº coef:", len(coef))

# 3) (Opcional) Mapear cada coeficiente al nombre de feature
#    Tomamos el metadata de 'features_lr' para obtener los nombres
tmp = best.transform(train.limit(1))
meta = tmp.schema["features_lr"].metadata.get("ml_attr", {})
attrs = []
for k in ("binary", "numeric"):  # OHE -> binary ; numéricas -> numeric
    if "attrs" in meta and k in meta["attrs"]:
        attrs += meta["attrs"][k]

feat_names = [a["name"] for a in attrs]
# puede haber un pequeño desfase si hay atributos sin nombre; recortamos al mínimo
n = min(len(feat_names), len(coef))
feat_names = feat_names[:n]
coef = coef[:n]

# Top-25 por magnitud
import pandas as pd
coef_df = pd.DataFrame({"feature": feat_names, "coef": coef})
coef_df["abs_coef"] = coef_df["coef"].abs()
print(coef_df.sort_values("abs_coef", ascending=False).head(25))



Intercept: 0.2634366115978087
Nº coef: 30
                          feature      coef  abs_coef
18                   avg_fact_imp  0.816328  0.816328
16         lag1_digital_ratio_imp -0.765888  0.765888
2      madurez_digital_cd_oh_ALTA  0.000000  0.000000
3     frecuencia_visitas_cd_oh_LM  0.000000  0.000000
0      madurez_digital_cd_oh_BAJA  0.000000  0.000000
1     madurez_digital_cd_oh_MEDIA  0.000000  0.000000
6    frecuencia_visitas_cd_oh_LMV  0.000000  0.000000
7                   pais_cd_oh_GT  0.000000  0.000000
8                   pais_cd_oh_EC  0.000000  0.000000
9                   pais_cd_oh_PE  0.000000  0.000000
10                  pais_cd_oh_SV  0.000000  0.000000
11      tipo_cliente_cd_oh_TIENDA  0.000000  0.000000
4      frecuencia_visitas_cd_oh_L  0.000000  0.000000
5    frecuencia_visitas_cd_oh_LMI  0.000000  0.000000
13   tipo_cliente_cd_oh_MAYORISTA  0.000000  0.000000
12  tipo_cliente_cd_oh_MINIMARKET  0.000000  0.000000
15              digital_ratio_imp  0.000

In [13]:
# Cierre ordenado de la sesión
spark.stop()
print("Spark session stopped.")

Spark session stopped.


---
### Notas
- Este cuaderno prioriza ranking (PR@k) y F1, más útiles que accuracy en desbalance.
- Puedes reducir `spark.sql.shuffle.partitions` y/o el grid para prototipos rápidos.
- Si ves OOM, baja cardinalidad de OHE (quita columnas con demasiadas categorías) o usa *hashing trick*.
- Para reporte: exporta la lista de clientes top-k con `p1` (score) y variables clave.